In [78]:
import csv
import pandas as pd
from pyblock2.driver.core import DMRGDriver, SymmetryTypes, MPOAlgorithmTypes
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm

# Data Storing Scripts

In [80]:
def write_to_csv(file_name, data):
    """
    Write data to a CSV file.

    Parameters:
    - file_name (str): The name of the CSV file to write to.
    - data (list of lists): The data to write to the CSV file, where each inner list is a row.
    """
    with open(file_name, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(data)

In [51]:
data = [ ['L', 't', 'U', 'N_bosons', 'N_max', 'GndEigenval', 'GndEigenvec', 'FirstExcitedEigenval', 'Entropy'] ]

In [52]:
write_to_csv("theta_PiBytwo_AHMDmrg_data.csv", data)

In [81]:
def add_data_to_csv(file_name, data, **kwargs):
    """
    Append new data to a CSV file using pandas.

    Parameters:
    - file_name (str): The name of the CSV file to write to.
    - data (list of lists): The data to append, where each sublist represents a row.
    """
    df = pd.DataFrame(data, columns=['L', 't', 'U', 'N_bosons', 'N_max', 'GndEigenval', 'GndEigenvec', 'FirstExcitedEigenval', 'Entropy'])
    
    df.to_csv(file_name, mode='a', index=False, header=False)

In [82]:
def delete_row_in_csv(file_name, **kwargs):
    """
    Delete rows in a CSV file based on a condition.

    Parameters:
    - file_name (str): The name of the CSV file to update.
    - condition (function): A function that takes a DataFrame row and returns True for rows to keep.
    """
    df = pd.read_csv(file_name)
    if 'condition' in kwargs:
        df = df[condition(df)]
    elif 'row_index' in kwargs:
        df = df.drop(kwargs['row_index'])
    df.to_csv(file_name, index=False)

# ## To delete first row
# delete_row_in_csv("theta0_AHMDmrg_data.csv", row_index = 0) 

# Data for $\theta = \frac{\pi}{2}$ for $L$ = 10, $N\_{Boson}$ = 10, $N\_{Max}$ = 3, $U$ = 1

In [83]:
params={}
params['L'] = 10
params['t'] = 1
params['u'] = 1
params['mu'] = 0
params['NB_MAX'] = 3 # max n_boson per site
params['N_BOSON'] = 10
params['theta'] = np.pi/2
params['theta_list'] = [0, np.pi/4, 3*np.pi/4, np.pi/2, np.pi]
params['t_list'] = np.arange(0.01, 2, 0.1)

driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SAny|SymmetryTypes.CPX, n_threads=4)

driver.set_symmetry_groups("U1")
Q = driver.bw.SX

MemoryError: bad allocation

In [56]:
site_basis, site_ops = [], []
for k in range(params['L']):
    basis = [(Q(i), 1) for i in range(params['NB_MAX'] + 1)] 
    ops = {
        "": np.identity(params['NB_MAX'] + 1),                           # identity
        "C": np.diag(np.sqrt(np.arange(1, params['NB_MAX'] + 1)), k=-1), # b+
        "D": np.diag(np.sqrt(np.arange(1, params['NB_MAX'] + 1)), k=1),  # b
        "N": np.diag(np.arange(0, params['NB_MAX'] + 1), k=0),           # particle number
        "A": np.diag(np.sqrt(np.arange(1, params['NB_MAX'] + 1))*np.exp(1j * params['theta'] * np.arange(params['NB_MAX'])), k=-1), # A+_withPhase  
        "B": np.diag(np.sqrt(np.arange(1, params['NB_MAX'] + 1))*np.exp(-1j * params['theta'] * np.arange(params['NB_MAX'])), k=1), # A_withPhase  
    }
    site_basis.append(basis)
    site_ops.append(ops)

## First Excited Gap and Entropy Data

#### $t = 0$

In [57]:
params['t'] = 0
driver.initialize_system(n_sites=params['L'], vacuum=Q(0), target=Q(params['N_BOSON']), hamil_init=False)
driver.ghamil = driver.get_custom_hamiltonian(site_basis, site_ops)
b = driver.expr_builder()

# b.add_term("ADCB", np.array([[i, i+1, i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("AD", np.array([[i, i+1] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("CB", np.array([[i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("N", np.array(np.arange(params['L'])), -(params['mu'] + params['u'] / 2))
b.add_term("NN", np.repeat(np.arange(params['L']), 2), params['u'] / 2)

mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
mps = driver.get_random_mps(tag="KET", bond_dim=50, nroots=2)
energy = driver.dmrg(mpo, mps, n_sweeps=10, bond_dims=[50] * 4 + [100] * 4,
    noises=[1e-4] * 4 + [1e-5] * 4 + [0], thrds=[1e-10] * 8, dav_max_iter=30, iprint=0)
entropy = driver.get_bipartite_entanglement()

In [58]:
#### Data Storing
data_t0_thetapibytwo = [[params['L'], params['t'], params['u'], params['N_BOSON'], params['NB_MAX'], energy[0], mps, energy[1], entropy]]
add_data_to_csv("theta_PiBytwo_AHMDmrg_data.csv", data_t0_thetapibytwo)

#### $t = 0.1$

In [59]:
params['t'] = 0.1
driver.initialize_system(n_sites=params['L'], vacuum=Q(0), target=Q(params['N_BOSON']), hamil_init=False)
driver.ghamil = driver.get_custom_hamiltonian(site_basis, site_ops)
b = driver.expr_builder()

# b.add_term("ADCB", np.array([[i, i+1, i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("AD", np.array([[i, i+1] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("CB", np.array([[i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("N", np.array(np.arange(params['L'])), -(params['mu'] + params['u'] / 2))
b.add_term("NN", np.repeat(np.arange(params['L']), 2), params['u'] / 2)

mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
mps = driver.get_random_mps(tag="KET", bond_dim=50, nroots=2)
energy = driver.dmrg(mpo, mps, n_sweeps=10, bond_dims=[50] * 4 + [100] * 4,
    noises=[1e-4] * 4 + [1e-5] * 4 + [0], thrds=[1e-10] * 8, dav_max_iter=30, iprint=0)
entropy = driver.get_bipartite_entanglement()

In [60]:
#### Data Storing
data_t0_1_thetapibytwo = [[params['L'], params['t'], params['u'], params['N_BOSON'], params['NB_MAX'], energy[0], mps, energy[1], entropy]]
add_data_to_csv("theta_PiBytwo_AHMDmrg_data.csv", data_t0_1_thetapibytwo)

#### $t = 0.2$

In [61]:
params['t'] = 0.2
driver.initialize_system(n_sites=params['L'], vacuum=Q(0), target=Q(params['N_BOSON']), hamil_init=False)
driver.ghamil = driver.get_custom_hamiltonian(site_basis, site_ops)
b = driver.expr_builder()

# b.add_term("ADCB", np.array([[i, i+1, i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("AD", np.array([[i, i+1] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("CB", np.array([[i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("N", np.array(np.arange(params['L'])), -(params['mu'] + params['u'] / 2))
b.add_term("NN", np.repeat(np.arange(params['L']), 2), params['u'] / 2)

mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
mps = driver.get_random_mps(tag="KET", bond_dim=50, nroots=2)
energy = driver.dmrg(mpo, mps, n_sweeps=10, bond_dims=[50] * 4 + [100] * 4,
    noises=[1e-4] * 4 + [1e-5] * 4 + [0], thrds=[1e-10] * 8, dav_max_iter=30, iprint=0)
entropy = driver.get_bipartite_entanglement()

In [62]:
#### Data Storing
data_t0_2_thetapibytwo = [[params['L'], params['t'], params['u'], params['N_BOSON'], params['NB_MAX'], energy[0], mps, energy[1], entropy]]
add_data_to_csv("theta_PiBytwo_AHMDmrg_data.csv", data_t0_2_thetapibytwo)

#### $t = 0.3$

In [63]:
params['t'] = 0.3
driver.initialize_system(n_sites=params['L'], vacuum=Q(0), target=Q(params['N_BOSON']), hamil_init=False)
driver.ghamil = driver.get_custom_hamiltonian(site_basis, site_ops)
b = driver.expr_builder()

# b.add_term("ADCB", np.array([[i, i+1, i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("AD", np.array([[i, i+1] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("CB", np.array([[i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("N", np.array(np.arange(params['L'])), -(params['mu'] + params['u'] / 2))
b.add_term("NN", np.repeat(np.arange(params['L']), 2), params['u'] / 2)

mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
mps = driver.get_random_mps(tag="KET", bond_dim=50, nroots=2)
energy = driver.dmrg(mpo, mps, n_sweeps=10, bond_dims=[50] * 4 + [100] * 4,
    noises=[1e-4] * 4 + [1e-5] * 4 + [0], thrds=[1e-10] * 8, dav_max_iter=30, iprint=0)
entropy = driver.get_bipartite_entanglement()

In [64]:
#### Data Storing
data_t0_3_thetapibytwo = [[params['L'], params['t'], params['u'], params['N_BOSON'], params['NB_MAX'], energy[0], mps, energy[1], entropy]]
add_data_to_csv("theta_PiBytwo_AHMDmrg_data.csv", data_t0_3_thetapibytwo)

#### $t = 0.4$

In [65]:
params['t'] = 0.4
driver.initialize_system(n_sites=params['L'], vacuum=Q(0), target=Q(params['N_BOSON']), hamil_init=False)
driver.ghamil = driver.get_custom_hamiltonian(site_basis, site_ops)
b = driver.expr_builder()

# b.add_term("ADCB", np.array([[i, i+1, i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("AD", np.array([[i, i+1] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("CB", np.array([[i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("N", np.array(np.arange(params['L'])), -(params['mu'] + params['u'] / 2))
b.add_term("NN", np.repeat(np.arange(params['L']), 2), params['u'] / 2)

mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
mps = driver.get_random_mps(tag="KET", bond_dim=50, nroots=2)
energy = driver.dmrg(mpo, mps, n_sweeps=10, bond_dims=[50] * 4 + [100] * 4,
    noises=[1e-4] * 4 + [1e-5] * 4 + [0], thrds=[1e-10] * 8, dav_max_iter=30, iprint=0)
entropy = driver.get_bipartite_entanglement()

In [66]:
#### Data Storing
data_t0_4_thetapibytwo = [[params['L'], params['t'], params['u'], params['N_BOSON'], params['NB_MAX'], energy[0], mps, energy[1], entropy]]
add_data_to_csv("theta_PiBytwo_AHMDmrg_data.csv", data_t0_4_thetapibytwo)

#### $t = 0.5$

In [67]:
params['t'] = 0.5
driver.initialize_system(n_sites=params['L'], vacuum=Q(0), target=Q(params['N_BOSON']), hamil_init=False)
driver.ghamil = driver.get_custom_hamiltonian(site_basis, site_ops)
b = driver.expr_builder()

# b.add_term("ADCB", np.array([[i, i+1, i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("AD", np.array([[i, i+1] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("CB", np.array([[i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("N", np.array(np.arange(params['L'])), -(params['mu'] + params['u'] / 2))
b.add_term("NN", np.repeat(np.arange(params['L']), 2), params['u'] / 2)

mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
mps = driver.get_random_mps(tag="KET", bond_dim=50, nroots=2)
energy = driver.dmrg(mpo, mps, n_sweeps=10, bond_dims=[50] * 4 + [100] * 4,
    noises=[1e-4] * 4 + [1e-5] * 4 + [0], thrds=[1e-10] * 8, dav_max_iter=30, iprint=0)
entropy = driver.get_bipartite_entanglement()

In [68]:
#### Data Storing
data_t0_5_thetapibytwo = [[params['L'], params['t'], params['u'], params['N_BOSON'], params['NB_MAX'], energy[0], mps, energy[1], entropy]]
add_data_to_csv("theta_PiBytwo_AHMDmrg_data.csv", data_t0_5_thetapibytwo)

#### $t = 0.6$

In [69]:
params['t'] = 0.6
driver.initialize_system(n_sites=params['L'], vacuum=Q(0), target=Q(params['N_BOSON']), hamil_init=False)
driver.ghamil = driver.get_custom_hamiltonian(site_basis, site_ops)
b = driver.expr_builder()

# b.add_term("ADCB", np.array([[i, i+1, i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("AD", np.array([[i, i+1] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("CB", np.array([[i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("N", np.array(np.arange(params['L'])), -(params['mu'] + params['u'] / 2))
b.add_term("NN", np.repeat(np.arange(params['L']), 2), params['u'] / 2)

mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
mps = driver.get_random_mps(tag="KET", bond_dim=50, nroots=2)
energy = driver.dmrg(mpo, mps, n_sweeps=10, bond_dims=[50] * 4 + [100] * 4,
    noises=[1e-4] * 4 + [1e-5] * 4 + [0], thrds=[1e-10] * 8, dav_max_iter=30, iprint=0)
entropy = driver.get_bipartite_entanglement()

In [70]:
#### Data Storing
data_t0_6_thetapibytwo = [[params['L'], params['t'], params['u'], params['N_BOSON'], params['NB_MAX'], energy[0], mps, energy[1], entropy]]
add_data_to_csv("theta_PiBytwo_AHMDmrg_data.csv", data_t0_6_thetapibytwo)

#### $t = 0.7$

In [71]:
params['t'] = 0.7
driver.initialize_system(n_sites=params['L'], vacuum=Q(0), target=Q(params['N_BOSON']), hamil_init=False)
driver.ghamil = driver.get_custom_hamiltonian(site_basis, site_ops)
b = driver.expr_builder()

# b.add_term("ADCB", np.array([[i, i+1, i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("AD", np.array([[i, i+1] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("CB", np.array([[i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("N", np.array(np.arange(params['L'])), -(params['mu'] + params['u'] / 2))
b.add_term("NN", np.repeat(np.arange(params['L']), 2), params['u'] / 2)

mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
mps = driver.get_random_mps(tag="KET", bond_dim=50, nroots=2)
energy = driver.dmrg(mpo, mps, n_sweeps=10, bond_dims=[50] * 4 + [100] * 4,
    noises=[1e-4] * 4 + [1e-5] * 4 + [0], thrds=[1e-10] * 8, dav_max_iter=30, iprint=0)
entropy = driver.get_bipartite_entanglement()

In [72]:
#### Data Storing
data_t0_7_thetapibytwo = [[params['L'], params['t'], params['u'], params['N_BOSON'], params['NB_MAX'], energy[0], mps, energy[1], entropy]]
add_data_to_csv("theta_PiBytwo_AHMDmrg_data.csv", data_t0_7_thetapibytwo)

#### $t = 0.8$

In [73]:
params['t'] = 0.8
driver.initialize_system(n_sites=params['L'], vacuum=Q(0), target=Q(params['N_BOSON']), hamil_init=False)
driver.ghamil = driver.get_custom_hamiltonian(site_basis, site_ops)
b = driver.expr_builder()

# b.add_term("ADCB", np.array([[i, i+1, i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("AD", np.array([[i, i+1] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("CB", np.array([[i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("N", np.array(np.arange(params['L'])), -(params['mu'] + params['u'] / 2))
b.add_term("NN", np.repeat(np.arange(params['L']), 2), params['u'] / 2)

mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
mps = driver.get_random_mps(tag="KET", bond_dim=50, nroots=2)
energy = driver.dmrg(mpo, mps, n_sweeps=10, bond_dims=[50] * 4 + [100] * 4,
    noises=[1e-4] * 4 + [1e-5] * 4 + [0], thrds=[1e-10] * 8, dav_max_iter=30, iprint=0)
entropy = driver.get_bipartite_entanglement()

MemoryError: bad allocation

In [ ]:
#### Data Storing
data_t0_8_thetapibytwo = [[params['L'], params['t'], params['u'], params['N_BOSON'], params['NB_MAX'], energy[0], mps, energy[1], entropy]]
add_data_to_csv("theta_PiBytwo_AHMDmrg_data.csv", data_t0_8_thetapibytwo)

#### $t = 0.9$

In [ ]:
params['t'] = 0.9
driver.initialize_system(n_sites=params['L'], vacuum=Q(0), target=Q(params['N_BOSON']), hamil_init=False)
driver.ghamil = driver.get_custom_hamiltonian(site_basis, site_ops)
b = driver.expr_builder()

# b.add_term("ADCB", np.array([[i, i+1, i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("AD", np.array([[i, i+1] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("CB", np.array([[i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("N", np.array(np.arange(params['L'])), -(params['mu'] + params['u'] / 2))
b.add_term("NN", np.repeat(np.arange(params['L']), 2), params['u'] / 2)

mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
mps = driver.get_random_mps(tag="KET", bond_dim=50, nroots=2)
energy = driver.dmrg(mpo, mps, n_sweeps=10, bond_dims=[50] * 4 + [100] * 4,
    noises=[1e-4] * 4 + [1e-5] * 4 + [0], thrds=[1e-10] * 8, dav_max_iter=30, iprint=0)
entropy = driver.get_bipartite_entanglement()

In [ ]:
#### Data Storing
data_t0_9_thetapibytwo = [[params['L'], params['t'], params['u'], params['N_BOSON'], params['NB_MAX'], energy[0], mps, energy[1], entropy]]
add_data_to_csv("theta_PiBytwo_AHMDmrg_data.csv", data_t0_9_thetapibytwo)

#### $t = 1.0$

In [ ]:
params['t'] = 1
driver.initialize_system(n_sites=params['L'], vacuum=Q(0), target=Q(params['N_BOSON']), hamil_init=False)
driver.ghamil = driver.get_custom_hamiltonian(site_basis, site_ops)
b = driver.expr_builder()

# b.add_term("ADCB", np.array([[i, i+1, i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("AD", np.array([[i, i+1] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("CB", np.array([[i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("N", np.array(np.arange(params['L'])), -(params['mu'] + params['u'] / 2))
b.add_term("NN", np.repeat(np.arange(params['L']), 2), params['u'] / 2)

mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
mps = driver.get_random_mps(tag="KET", bond_dim=50, nroots=2)
energy = driver.dmrg(mpo, mps, n_sweeps=10, bond_dims=[50] * 4 + [100] * 4,
    noises=[1e-4] * 4 + [1e-5] * 4 + [0], thrds=[1e-10] * 8, dav_max_iter=30, iprint=0)
entropy = driver.get_bipartite_entanglement()

In [ ]:
#### Data Storing
data_t1_thetapibytwo = [[params['L'], params['t'], params['u'], params['N_BOSON'], params['NB_MAX'], energy[0], mps, energy[1], entropy]]
add_data_to_csv("theta_PiBytwo_AHMDmrg_data.csv", data_t1_thetapibytwo)

#### $t = 1.1$

In [ ]:
params['t'] = 1.1
driver.initialize_system(n_sites=params['L'], vacuum=Q(0), target=Q(params['N_BOSON']), hamil_init=False)
driver.ghamil = driver.get_custom_hamiltonian(site_basis, site_ops)
b = driver.expr_builder()

# b.add_term("ADCB", np.array([[i, i+1, i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("AD", np.array([[i, i+1] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("CB", np.array([[i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("N", np.array(np.arange(params['L'])), -(params['mu'] + params['u'] / 2))
b.add_term("NN", np.repeat(np.arange(params['L']), 2), params['u'] / 2)

mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
mps = driver.get_random_mps(tag="KET", bond_dim=50, nroots=2)
energy = driver.dmrg(mpo, mps, n_sweeps=10, bond_dims=[50] * 4 + [100] * 4,
    noises=[1e-4] * 4 + [1e-5] * 4 + [0], thrds=[1e-10] * 8, dav_max_iter=30, iprint=0)
entropy = driver.get_bipartite_entanglement()

In [ ]:
#### Data Storing
data_t1_1_thetapibytwo = [[params['L'], params['t'], params['u'], params['N_BOSON'], params['NB_MAX'], energy[0], mps, energy[1], entropy]]
add_data_to_csv("theta_PiBytwo_AHMDmrg_data.csv", data_t1_1_thetapibytwo)

#### $t = 1.2$

In [ ]:
params['t'] = 1.2
driver.initialize_system(n_sites=params['L'], vacuum=Q(0), target=Q(params['N_BOSON']), hamil_init=False)
driver.ghamil = driver.get_custom_hamiltonian(site_basis, site_ops)
b = driver.expr_builder()

# b.add_term("ADCB", np.array([[i, i+1, i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("AD", np.array([[i, i+1] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("CB", np.array([[i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("N", np.array(np.arange(params['L'])), -(params['mu'] + params['u'] / 2))
b.add_term("NN", np.repeat(np.arange(params['L']), 2), params['u'] / 2)

mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
mps = driver.get_random_mps(tag="KET", bond_dim=50, nroots=2)
energy = driver.dmrg(mpo, mps, n_sweeps=10, bond_dims=[50] * 4 + [100] * 4,
    noises=[1e-4] * 4 + [1e-5] * 4 + [0], thrds=[1e-10] * 8, dav_max_iter=30, iprint=0)
entropy = driver.get_bipartite_entanglement()

In [ ]:
#### Data Storing
data_t1_2_thetapibytwo = [[params['L'], params['t'], params['u'], params['N_BOSON'], params['NB_MAX'], energy[0], mps, energy[1], entropy]]
add_data_to_csv("theta_PiBytwo_AHMDmrg_data.csv", data_t1_2_thetapibytwo)

#### $t = 1.3$

In [ ]:
params['t'] = 1.3
driver.initialize_system(n_sites=params['L'], vacuum=Q(0), target=Q(params['N_BOSON']), hamil_init=False)
driver.ghamil = driver.get_custom_hamiltonian(site_basis, site_ops)
b = driver.expr_builder()

# b.add_term("ADCB", np.array([[i, i+1, i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("AD", np.array([[i, i+1] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("CB", np.array([[i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("N", np.array(np.arange(params['L'])), -(params['mu'] + params['u'] / 2))
b.add_term("NN", np.repeat(np.arange(params['L']), 2), params['u'] / 2)

mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
mps = driver.get_random_mps(tag="KET", bond_dim=50, nroots=2)
energy = driver.dmrg(mpo, mps, n_sweeps=10, bond_dims=[50] * 4 + [100] * 4,
    noises=[1e-4] * 4 + [1e-5] * 4 + [0], thrds=[1e-10] * 8, dav_max_iter=30, iprint=0)
entropy = driver.get_bipartite_entanglement()

In [ ]:
#### Data Storing
data_t1_3_thetapibytwo = [[params['L'], params['t'], params['u'], params['N_BOSON'], params['NB_MAX'], energy[0], mps, energy[1], entropy]]
add_data_to_csv("theta_PiBytwo_AHMDmrg_data.csv", data_t1_3_thetapibytwo)

#### $t = 1.4$

In [ ]:
params['t'] = 1.4
driver.initialize_system(n_sites=params['L'], vacuum=Q(0), target=Q(params['N_BOSON']), hamil_init=False)
driver.ghamil = driver.get_custom_hamiltonian(site_basis, site_ops)
b = driver.expr_builder()

# b.add_term("ADCB", np.array([[i, i+1, i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("AD", np.array([[i, i+1] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("CB", np.array([[i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("N", np.array(np.arange(params['L'])), -(params['mu'] + params['u'] / 2))
b.add_term("NN", np.repeat(np.arange(params['L']), 2), params['u'] / 2)

mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
mps = driver.get_random_mps(tag="KET", bond_dim=50, nroots=2)
energy = driver.dmrg(mpo, mps, n_sweeps=10, bond_dims=[50] * 4 + [100] * 4,
    noises=[1e-4] * 4 + [1e-5] * 4 + [0], thrds=[1e-10] * 8, dav_max_iter=30, iprint=0)
entropy = driver.get_bipartite_entanglement()

In [ ]:
#### Data Storing
data_t1_4_thetapibytwo = [[params['L'], params['t'], params['u'], params['N_BOSON'], params['NB_MAX'], energy[0], mps, energy[1], entropy]]
add_data_to_csv("theta_PiBytwo_AHMDmrg_data.csv", data_t1_4_thetapibytwo)

#### $t = 1.5$

In [ ]:
params['t'] = 1.5
driver.initialize_system(n_sites=params['L'], vacuum=Q(0), target=Q(params['N_BOSON']), hamil_init=False)
driver.ghamil = driver.get_custom_hamiltonian(site_basis, site_ops)
b = driver.expr_builder()

# b.add_term("ADCB", np.array([[i, i+1, i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("AD", np.array([[i, i+1] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("CB", np.array([[i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("N", np.array(np.arange(params['L'])), -(params['mu'] + params['u'] / 2))
b.add_term("NN", np.repeat(np.arange(params['L']), 2), params['u'] / 2)

mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
mps = driver.get_random_mps(tag="KET", bond_dim=50, nroots=2)
energy = driver.dmrg(mpo, mps, n_sweeps=10, bond_dims=[50] * 4 + [100] * 4,
    noises=[1e-4] * 4 + [1e-5] * 4 + [0], thrds=[1e-10] * 8, dav_max_iter=30, iprint=0)
entropy = driver.get_bipartite_entanglement()

In [ ]:
#### Data Storing
data_t1_5_thetapibytwo = [[params['L'], params['t'], params['u'], params['N_BOSON'], params['NB_MAX'], energy[0], mps, energy[1], entropy]]
add_data_to_csv("theta_PiBytwo_AHMDmrg_data.csv", data_t1_5_thetapibytwo)

#### $t = 1.6$

In [ ]:
params['t'] = 1.6
driver.initialize_system(n_sites=params['L'], vacuum=Q(0), target=Q(params['N_BOSON']), hamil_init=False)
driver.ghamil = driver.get_custom_hamiltonian(site_basis, site_ops)
b = driver.expr_builder()

# b.add_term("ADCB", np.array([[i, i+1, i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("AD", np.array([[i, i+1] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("CB", np.array([[i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("N", np.array(np.arange(params['L'])), -(params['mu'] + params['u'] / 2))
b.add_term("NN", np.repeat(np.arange(params['L']), 2), params['u'] / 2)

mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
mps = driver.get_random_mps(tag="KET", bond_dim=50, nroots=2)
energy = driver.dmrg(mpo, mps, n_sweeps=10, bond_dims=[50] * 4 + [100] * 4,
    noises=[1e-4] * 4 + [1e-5] * 4 + [0], thrds=[1e-10] * 8, dav_max_iter=30, iprint=0)
entropy = driver.get_bipartite_entanglement()

In [ ]:
#### Data Storing
data_t1_6_thetapibytwo = [[params['L'], params['t'], params['u'], params['N_BOSON'], params['NB_MAX'], energy[0], mps, energy[1], entropy]]
add_data_to_csv("theta_PiBytwo_AHMDmrg_data.csv", data_t1_6_thetapibytwo)

#### $t = 1.7$

In [ ]:
params['t'] = 1.7
driver.initialize_system(n_sites=params['L'], vacuum=Q(0), target=Q(params['N_BOSON']), hamil_init=False)
driver.ghamil = driver.get_custom_hamiltonian(site_basis, site_ops)
b = driver.expr_builder()

# b.add_term("ADCB", np.array([[i, i+1, i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("AD", np.array([[i, i+1] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("CB", np.array([[i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("N", np.array(np.arange(params['L'])), -(params['mu'] + params['u'] / 2))
b.add_term("NN", np.repeat(np.arange(params['L']), 2), params['u'] / 2)

mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
mps = driver.get_random_mps(tag="KET", bond_dim=50, nroots=2)
energy = driver.dmrg(mpo, mps, n_sweeps=10, bond_dims=[50] * 4 + [100] * 4,
    noises=[1e-4] * 4 + [1e-5] * 4 + [0], thrds=[1e-10] * 8, dav_max_iter=30, iprint=0)
entropy = driver.get_bipartite_entanglement()

In [ ]:
#### Data Storing
data_t1_7_thetapibytwo = [[params['L'], params['t'], params['u'], params['N_BOSON'], params['NB_MAX'], energy[0], mps, energy[1], entropy]]
add_data_to_csv("theta_PiBytwo_AHMDmrg_data.csv", data_t1_7_thetapibytwo)

#### $t = 1.8$

In [ ]:
params['t'] = 1.8
driver.initialize_system(n_sites=params['L'], vacuum=Q(0), target=Q(params['N_BOSON']), hamil_init=False)
driver.ghamil = driver.get_custom_hamiltonian(site_basis, site_ops)
b = driver.expr_builder()

# b.add_term("ADCB", np.array([[i, i+1, i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("AD", np.array([[i, i+1] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("CB", np.array([[i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("N", np.array(np.arange(params['L'])), -(params['mu'] + params['u'] / 2))
b.add_term("NN", np.repeat(np.arange(params['L']), 2), params['u'] / 2)

mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
mps = driver.get_random_mps(tag="KET", bond_dim=50, nroots=2)
energy = driver.dmrg(mpo, mps, n_sweeps=10, bond_dims=[50] * 4 + [100] * 4,
    noises=[1e-4] * 4 + [1e-5] * 4 + [0], thrds=[1e-10] * 8, dav_max_iter=30, iprint=0)
entropy = driver.get_bipartite_entanglement()

In [ ]:
#### Data Storing
data_t1_8_thetapibytwo = [[params['L'], params['t'], params['u'], params['N_BOSON'], params['NB_MAX'], energy[0], mps, energy[1], entropy]]
add_data_to_csv("theta_PiBytwo_AHMDmrg_data.csv", data_t1_8_thetapibytwo)

#### $t = 1.9$

In [ ]:
params['t'] = 1.9
driver.initialize_system(n_sites=params['L'], vacuum=Q(0), target=Q(params['N_BOSON']), hamil_init=False)
driver.ghamil = driver.get_custom_hamiltonian(site_basis, site_ops)
b = driver.expr_builder()

# b.add_term("ADCB", np.array([[i, i+1, i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("AD", np.array([[i, i+1] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("CB", np.array([[i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("N", np.array(np.arange(params['L'])), -(params['mu'] + params['u'] / 2))
b.add_term("NN", np.repeat(np.arange(params['L']), 2), params['u'] / 2)

mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
mps = driver.get_random_mps(tag="KET", bond_dim=50, nroots=2)
energy = driver.dmrg(mpo, mps, n_sweeps=10, bond_dims=[50] * 4 + [100] * 4,
    noises=[1e-4] * 4 + [1e-5] * 4 + [0], thrds=[1e-10] * 8, dav_max_iter=30, iprint=0)
entropy = driver.get_bipartite_entanglement()

In [ ]:
#### Data Storing
data_t1_9_thetapibytwo = [[params['L'], params['t'], params['u'], params['N_BOSON'], params['NB_MAX'], energy[0], mps, energy[1], entropy]]
add_data_to_csv("theta_PiBytwo_AHMDmrg_data.csv", data_t1_9_thetapibytwo)

#### $t = 2.0$

In [ ]:
params['t'] = 2
driver.initialize_system(n_sites=params['L'], vacuum=Q(0), target=Q(params['N_BOSON']), hamil_init=False)
driver.ghamil = driver.get_custom_hamiltonian(site_basis, site_ops)
b = driver.expr_builder()

# b.add_term("ADCB", np.array([[i, i+1, i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("AD", np.array([[i, i+1] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("CB", np.array([[i+1, i] for i in range(params['L']-1)]).flatten(), -params['t'])
b.add_term("N", np.array(np.arange(params['L'])), -(params['mu'] + params['u'] / 2))
b.add_term("NN", np.repeat(np.arange(params['L']), 2), params['u'] / 2)

mpo = driver.get_mpo(b.finalize(adjust_order=True, fermionic_ops=""), algo_type=MPOAlgorithmTypes.FastBipartite)
mps = driver.get_random_mps(tag="KET", bond_dim=50, nroots=2)
energy = driver.dmrg(mpo, mps, n_sweeps=10, bond_dims=[50] * 4 + [100] * 4,
    noises=[1e-4] * 4 + [1e-5] * 4 + [0], thrds=[1e-10] * 8, dav_max_iter=30, iprint=0)
entropy = driver.get_bipartite_entanglement()

In [ ]:
#### Data Storing
data_t2_thetapibytwo = [[params['L'], params['t'], params['u'], params['N_BOSON'], params['NB_MAX'], energy[0], mps, energy[1], entropy]]
add_data_to_csv("theta_PiBytwo_AHMDmrg_data.csv", data_t2_thetapibytwo)